# Class Notes

## Bijectors Intro

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
z = tf.constant([1., 2., 3.])
scale = tfb.Scale(2.)

x = scale.forward(z)
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 6.], dtype=float32)>

In [3]:
scale.inverse(tf.constant([5., 3., 1.])) # ie x / scale

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.5, 1.5, 0.5], dtype=float32)>

### Shift

In [4]:
z = tf.constant([1., 2., 3.])
scale = tfb.Scale(2.)
shift = tfb.Shift(1.)

scale_and_shift = tfb.Chain([shift, scale])
scale_and_shift

In [5]:
scale_and_shift.forward(z)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([3., 5., 7.], dtype=float32)>

In [6]:
scale_and_shift.inverse(z) # Subtract and Scale(divide)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0. , 0.5, 1. ], dtype=float32)>

### Transform Random Variables

In [7]:
normal = tfd.Normal(loc=0, scale=1.)
z = normal.sample(3)
z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.34980565,  0.6862305 , -0.70671827], dtype=float32)>

In [8]:
scale_and_shift = tfb.Chain([tfb.Shift(1.), tfb.Scale(2.)])
scale_and_shift

In [9]:
x = scale_and_shift.forward(z)
x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.3003887 ,  2.3724608 , -0.41343653], dtype=float32)>

In [10]:
log_prob_z = normal.log_prob(z)
log_prob_z

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.98012054, -1.1543946 , -1.1686639 ], dtype=float32)>

In [11]:
# Change of variable formula
log_prob_x = normal.log_prob(z) - scale_and_shift.forward_log_det_jacobian(z, event_ndims=0)
log_prob_x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.6732677, -1.8475418, -1.861811 ], dtype=float32)>

## The Tranformed Distributation Class

In [14]:
log_prob_x = (normal.log_prob(scale_and_shift.inverse(x)) + scale_and_shift.inverse_log_det_jacobian(x, event_ndims=0))
log_prob_x

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.6732677, -1.8475418, -1.861811 ], dtype=float32)>

In [16]:
normal = tfd.Normal(loc=0, scale=1.)
z = normal.sample(3)

exp = tfb.Exp()
x = exp.forward(z)

In [18]:
log_normal = tfd.TransformedDistribution(normal, exp)
log_normal

<tfp.distributions.TransformedDistribution 'expNormal' batch_shape=[] event_shape=[] dtype=float32>

In [19]:
log_normal = exp(normal)
log_normal

<tfp.distributions.TransformedDistribution 'expNormal' batch_shape=[] event_shape=[] dtype=float32>

In [20]:
normal = tfd.Normal(loc=0, scale=1.)
scale_tril = [[[1.,0], [1., 1.]], [[0.5, 0], [-1, 0.5]]]
scale = tfb.ScaleMatvecTriL(scale_tril=scale_tril)

In [21]:
mvn = tfd.TransformedDistribution(normal, scale, batch_shape=[2], event_shape=[2])

TypeError: __init__() got an unexpected keyword argument 'batch_shape'

In [23]:
# Quiz

normal = tfd.Normal(loc=0., scale=[1., 1.])
scale = tfb.Scale([1., 2.])
mvn = tfd.TransformedDistribution(normal, scale)
mvn

<tfp.distributions.TransformedDistribution 'scaleNormal' batch_shape=[2] event_shape=[] dtype=float32>

## Subclassing Bijectors

In [24]:
class MySigmoid(tfb.Bijector):
    
    def __init__(self, validate_args=False, name='sigmoid'):
        super(MySigmoid, self).__init__(
            validate_args=validate_args,
            forward_min_event_ndims=0,
            name=name,
            is_constant_jacobian=True
        )
        
    def _forward(self, x):
        return tf.math.sigmoid(x)
    
    def _inverse(self, y):
        return tf.math.log(y) - tf.math.log(1-y)
    
    def _inverse_log_det_jacobian(self, y):
        return -tf.math.log(y) - tf.math.log(1-y)
    
#     def _forward_log_det_jacobian(self, x):
#         return -tf.math.softplus(-x) - tf.math.softplus(x)

    def _forward_log_det_jacobian(self, x):
        return -self._inverse_log_det_jacobian(self.forward(x))